
### Entrada
- x1: Local da Coleta em km
- x2: Nivel da Mare em m
- x3: Umidade do Ar (%)
- x4: Índice Pluvimétrico (mm)¹
- x5: Temperatura (°C)

### Saida
- y1: pH
- y2: Condutividade Elétrica (mS.cmˉ¹)
- y3: Oxigênio Dissolvido (mg.Lˉ¹)
- y4: Sólidos Totais Dissolvidos (ppm)


In [1]:
!pip install --upgrade pip
!pip install tensorflow 
!pip install openpyxl

In [2]:
import os

os.chdir("./content")

lm_dir = "tf-levenberg-marquardt"
if not os.path.exists(lm_dir):
  !git clone https://github.com/fabiodimarco/$lm_dir

os.chdir(lm_dir)

In [3]:
import numpy as np
import pandas as pd

def create_dataframe(output, label):
    df = pd.read_excel("../../../data.xlsx")
    x1, x2, x3, x4, x5  = df['x1'], df['x2'], df['x3'], df['x4'], df['x5']

    input = np.vstack([x1, x2, x3, x4, x5 ]).T
    output = np.array(df[output])
    return output, input

output, input = create_dataframe(output="y1", label="ph")

/tmp/ipykernel_177595/4275043310.py:2: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [4]:
import tensorflow as tf
import numpy as np
from keras import regularizers
from keras import initializers
import levenberg_marquardt as lm

# layers, neurons
class ShuffleArchitecture:
    def __init__(self, input_size, hidden_sizes, output_size, act_h, act_o, param_reg):
        self.input_size = input_size
        self.hidden_sizes = hidden_sizes
        self.output_size = output_size
        self.act_h = act_h
        self.act_o = act_o
        self.regularizer = regularizers.L2(param_reg)
        self.initializer = initializers.RandomUniform(minval=-0.5, maxval=0.5, seed=np.random.randint(1, 10000))

    def set_architecture(self):
        self.model = tf.keras.Sequential()
        self.model.add(tf.keras.layers.Dense(self.hidden_sizes[0],
                        input_shape=(self.input_size,),
                        activation=self.act_h,
                        kernel_regularizer=self.regularizer,
                        kernel_initializer=self.initializer,                        
                        ))  # input layer
        
        for size in self.hidden_sizes[1:]:  # hidden layers
            self.model.add(tf.keras.layers.Dense(size,
                            activation=self.act_h,
                            kernel_regularizer=self.regularizer,
                            kernel_initializer=self.initializer,  
                        ))

        self.model.add(tf.keras.layers.Dense(self.output_size,
                        activation=self.act_o,
                        kernel_regularizer=self.regularizer,
                        kernel_initializer=self.initializer,  
                        ))  # output layer

    def create_model(self, _learning_rate):
        self.model.compile(
            optimizer=tf.keras.optimizers.Adam(learning_rate=_learning_rate),
            loss=tf.keras.losses.MeanSquaredError())

        self.lm_model = lm.ModelWrapper(
            tf.keras.models.clone_model(self.model))

        self.lm_model.compile(
            optimizer=tf.keras.optimizers.SGD(learning_rate=_learning_rate),
            loss=lm.MeanSquaredError())
        return(self.lm_model)

2024-03-24 03:04:03.677627: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-03-24 03:04:03.755338: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-03-24 03:04:03.755382: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-03-24 03:04:03.756656: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-03-24 03:04:03.764983: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-03-24 03:04:03.766315: I tensorflow/core/platform/cpu_feature_guard.cc:1

In [5]:
import tensorflow as tf
import matplotlib.pyplot as plt
import time
from sklearn.model_selection import train_test_split
from keras.callbacks import EarlyStopping
from sklearn.metrics import r2_score, mean_squared_error


class TrainWithSmallDataset:
    def __init__(self, batch_size=1000):
        self.batch_size = batch_size
        self.betters = []
    
    def create_dataset(self, input, output):
      input = tf.expand_dims(tf.cast(input, tf.float32), axis=-1)
      output = tf.expand_dims(tf.cast(output, tf.float32), axis=-1)

      dataset = tf.data.Dataset.from_tensor_slices((input, output))
      dataset = dataset.shuffle(len(input))
      dataset = dataset.batch(self.batch_size).cache()
      dataset = dataset.prefetch(tf.data.experimental.AUTOTUNE)
      return (dataset, input, output)

    def split_dataset(self, input, output):
      input_train, input_vt, output_train, output_vt = train_test_split(input, output, test_size=0.3, shuffle = True)
      input_val, input_test, output_val, output_test = train_test_split(input_vt, output_vt, test_size=0.5, shuffle = True)

      self.train_dataset, self.train_input, self.train_output = self.create_dataset(input_train, output_train)
      self.val_dataset, self.val_input, self.val_output = self.create_dataset(input_val, output_val)
      self.test_dataset, self.test_input, self.test_output = self.create_dataset(input_test, output_test)
      self.vt_dataset, self.vt_input, self.vt_output = self.create_dataset(input_vt, output_vt)
      self.dataset, self.input, self.output = self.create_dataset(input, output)

      self._train = (input_train, output_train)
      self._vt = (input_vt, output_vt)
      self._val = (input_val, output_val)
      self._test = (input_test, output_test)
 
 
    
    def train_using_lm(self, train_dataset, epochs=1000):
      early_stopping_monitor = EarlyStopping(monitor='val_loss',
                                              patience=6,
                                              restore_best_weights=True)

      t2_start = time.perf_counter()
      self.results = self.lm_model.fit(train_dataset,
                                            epochs=epochs,
                                            validation_data=self.val_dataset,
                                            callbacks=[early_stopping_monitor],
                                            verbose=0)
      t2_stop = time.perf_counter()
      print("Elapsed time: ", t2_stop - t2_start)
      print ("Stopped at epoch: ", early_stopping_monitor.stopped_epoch)

    def get_metrics(self):
          self.test_prediction = self.lm_model.predict(self.test_input)

          pred = self.lm_model.predict(self.input).flatten()
          test_pred = self.test_prediction.flatten()
          val_pred = self.lm_model.predict(self.val_input).flatten()
          vt_pred = self.lm_model.predict(self.vt_input).flatten()

          r2 = r2_score(self.output, pred)
          r2_test = r2_score(self.test_output, test_pred)
          r2_val = r2_score(self.val_output, val_pred)
          r2_vt = r2_score(self.vt_output, vt_pred)

          mse = mean_squared_error(self.output, pred)
          mse_test = mean_squared_error(self.test_output, test_pred)
          mse_val = mean_squared_error(self.val_output, val_pred)
          mse_vt = mean_squared_error(self.vt_output, vt_pred)

          metrics = {
                          'r2': r2,
                          'r2_test': r2_test,
                          'r2_val': r2_val,
                          'r2_vt': r2_vt,
                          'mse': mse,
                          'mse_test': mse_test,
                          'mse_val': mse_val,
                          'mse_vt': mse_vt
                          }

          return metrics

In [6]:
from itertools import product
import pickle

class Tester:
  def __init__(self, input, output, run_times=500, dataset_run_times=10):
    self.run_times = run_times
    self.better_metrics = {}
    self.dataset_run_times = dataset_run_times
    self.input, self.output = input, output
  
  def setArchitecure(self, trainer, _hidden_sizes, _pg, _lr):
    shuffler = ShuffleArchitecture(input_size=5,
                                    hidden_sizes=_hidden_sizes,
                                    output_size=1,
                                    act_h='tanh',
                                    act_o='linear',
                                    param_reg=_pg)
    shuffler.set_architecture()    
    trainer.lm_model = shuffler.create_model(_lr)

  def Train(self, trainer, epochs=1000):
    trainer.train_using_lm(trainer.train_dataset, epochs=epochs)
    return(trainer.get_metrics(), trainer.lm_model)

  def SaveModelWeights(self, model, fileName):
    path = f"../models/{fileName}.keras"
    open(path,'w').close()
    model.save_weights(path) 

  def SaveDataset(self, trainer, fileName):
    path = f"../dataset/{fileName}.pkl" 
    with open(path, 'wb') as f:
      pickle.dump((trainer._train, trainer._vt, trainer._val, trainer._test), f)


  def LoopWeights(self, sort_by, boundarie, trainer, idx):
    better_model = 0
    save = False

    for i in range(self.run_times):
      print (f"+++++++++++ [{idx}] | {i + 1} ++++++++++++++++++")
      metrics, model = self.Train(trainer)
      if (metrics[sort_by] >= boundarie): # should be <= to descending metrics
        fileName = f"model_{idx}_{better_model}"
        self.SaveModelWeights(model, fileName)
        self.better_metrics[fileName] = metrics
        better_model += 1
        save = True
    
    return(save)

# Init
  def Loop(self, sort_by, boundarie, hidden_sizes, regularizers, learning_rate):
    trainer = TrainWithSmallDataset()

    for count, (hidden_size, reg, lr) in enumerate(product(hidden_sizes, regularizers, learning_rate), start=1):
      header =  f"Hidden Size={hidden_size}, regularizer={reg}, learning_rate={lr}"
      print(f"Testando combinacao{count}: {header}")
      self.setArchitecure(trainer, hidden_size, reg, lr)
      for j in range(self.dataset_run_times):
        trainer.split_dataset(self.input, self.output)
        if (self.LoopWeights(sort_by, boundarie, trainer, f"{count}_{j}") == True):
          self.SaveDataset(trainer, f"dataset_{count}_{j}")
          self.DisplayBetterResults(sort_by, header, f"{count}_{j}")
        self.better_metrics = {}

  def DisplayBetterResults(self, sort_by, header, dataset=0):
    df = pd.DataFrame.from_dict(self.better_metrics, orient='index')
    df = df.sort_values([sort_by])
    display(df)
    path = f'../results/metrics_{dataset}'
    df.to_excel(f"{path}.xlsx", index=True)
    print(f"DataFrame salvo em {path}")
    with open(f"{path}.txt", 'w') as arquivo:
      arquivo.write(header)

In [7]:
tester = Tester(input, output, run_times=25, dataset_run_times=10)

In [8]:
tester.Loop(sort_by='r2',
            boundarie=0.8,
            hidden_sizes = [[5], [10], [15], [20], [25], [30], [35], [40], [45]],
            regularizers=[0.01, 0.1],
            learning_rate=[0.1])

Testando combinacao1: Hidden Size=[5], regularizer=0.01, learning_rate=0.1
+++++++++++ [1_0] | 1 ++++++++++++++++++
Elapsed time:  2.1893816799997694
Stopped at epoch:  15
1/1 [==============================] - 0s 29ms/step
+++++++++++ [1_0] | 2 ++++++++++++++++++
Elapsed time:  0.8581124460001774
Stopped at epoch:  13
1/1 [==============================] - 0s 32ms/step
+++++++++++ [1_0] | 3 ++++++++++++++++++
Elapsed time:  0.5110008019996712
Stopped at epoch:  6
1/1 [==============================] - 0s 44ms/step
+++++++++++ [1_0] | 4 ++++++++++++++++++
Elapsed time:  0.728439100000287
Stopped at epoch:  11
1/1 [==============================] - 0s 35ms/step
+++++++++++ [1_0] | 5 ++++++++++++++++++
Elapsed time:  0.9210929239998222
Stopped at epoch:  13
1/1 [==============================] - 0s 24ms/step
+++++++++++ [1_0] | 6 ++++++++++++++++++
Elapsed time:  0.43701403599970945
Stopped at epoch:  7
1/1 [==============================] - 0s 41ms/step
+++++++++++ [1_0] | 7 +++++++++++

2024-03-24 03:19:09.088169: W tensorflow/core/data/root_dataset.cc:342] Optimization loop failed: CANCELLED: Operation was cancelled


Elapsed time:  0.525573273000191
Stopped at epoch:  6
1/1 [==============================] - 0s 26ms/step
+++++++++++ [6_4] | 24 ++++++++++++++++++
Elapsed time:  0.4732501290000073
Stopped at epoch:  6
1/1 [==============================] - 0s 87ms/step
+++++++++++ [6_4] | 25 ++++++++++++++++++
Elapsed time:  0.7479367239998282
Stopped at epoch:  6
1/1 [==============================] - 0s 23ms/step
+++++++++++ [6_5] | 1 ++++++++++++++++++
Elapsed time:  0.9563734359999216
Stopped at epoch:  10
1/1 [==============================] - 0s 24ms/step
+++++++++++ [6_5] | 2 ++++++++++++++++++
Elapsed time:  0.5956353429996852
Stopped at epoch:  6
1/1 [==============================] - 0s 63ms/step
+++++++++++ [6_5] | 3 ++++++++++++++++++
Elapsed time:  0.613520385999891
Stopped at epoch:  6
1/1 [==============================] - 0s 30ms/step
+++++++++++ [6_5] | 4 ++++++++++++++++++
Elapsed time:  0.5893122810002751
Stopped at epoch:  6
1/1 [==============================] - 0s 28ms/step
++++

2024-03-24 03:20:44.911973: W tensorflow/core/data/root_dataset.cc:342] Optimization loop failed: CANCELLED: Operation was cancelled


Elapsed time:  0.6281088779996935
Stopped at epoch:  6
1/1 [==============================] - 0s 34ms/step
+++++++++++ [6_8] | 4 ++++++++++++++++++
Elapsed time:  0.7157697550001103
Stopped at epoch:  6
1/1 [==============================] - 0s 40ms/step
+++++++++++ [6_8] | 5 ++++++++++++++++++
Elapsed time:  0.6952121819999775
Stopped at epoch:  6
1/1 [==============================] - 0s 43ms/step
+++++++++++ [6_8] | 6 ++++++++++++++++++
Elapsed time:  0.7722422030001326
Stopped at epoch:  6
1/1 [==============================] - 0s 27ms/step
+++++++++++ [6_8] | 7 ++++++++++++++++++
Elapsed time:  0.6898228420000123
Stopped at epoch:  6
1/1 [==============================] - 0s 25ms/step
+++++++++++ [6_8] | 8 ++++++++++++++++++
Elapsed time:  0.6253256260001763
Stopped at epoch:  6
1/1 [==============================] - 0s 25ms/step
+++++++++++ [6_8] | 9 ++++++++++++++++++
Elapsed time:  0.7550458620003155
Stopped at epoch:  6
1/1 [==============================] - 0s 19ms/step
+++++

...

DataFrame salvo em ../results/metrics_18_4
+++++++++++ [18_5] | 1 ++++++++++++++++++


2024-03-24 03:56:12.321897: W tensorflow/core/data/root_dataset.cc:342] Optimization loop failed: CANCELLED: Operation was cancelled


Elapsed time:  0.3789701719997538
Stopped at epoch:  7
1/1 [==============================] - 0s 26ms/step
+++++++++++ [18_5] | 2 ++++++++++++++++++
Elapsed time:  0.4979198399996676
Stopped at epoch:  6
1/1 [==============================] - 0s 27ms/step
+++++++++++ [18_5] | 3 ++++++++++++++++++
Elapsed time:  0.33325603699995554
Stopped at epoch:  6
1/1 [==============================] - 0s 24ms/step
+++++++++++ [18_5] | 4 ++++++++++++++++++
Elapsed time:  0.31307137700059684
Stopped at epoch:  6
1/1 [==============================] - 0s 26ms/step
+++++++++++ [18_5] | 5 ++++++++++++++++++
Elapsed time:  0.2744867729998077
Stopped at epoch:  6
1/1 [==============================] - 0s 21ms/step
+++++++++++ [18_5] | 6 ++++++++++++++++++
Elapsed time:  0.31116660099996807
Stopped at epoch:  6
1/1 [==============================] - 0s 23ms/step
+++++++++++ [18_5] | 7 ++++++++++++++++++
Elapsed time:  0.3492981939998572
Stopped at epoch:  6
1/1 [==============================] - 0s 23ms/s

2024-03-24 03:57:34.997327: W tensorflow/core/data/root_dataset.cc:342] Optimization loop failed: CANCELLED: Operation was cancelled


Elapsed time:  0.4830847069997617
Stopped at epoch:  6
1/1 [==============================] - 0s 22ms/step
+++++++++++ [18_9] | 22 ++++++++++++++++++
Elapsed time:  0.31414059400049155
Stopped at epoch:  6
1/1 [==============================] - 0s 23ms/step
+++++++++++ [18_9] | 23 ++++++++++++++++++
Elapsed time:  0.26178070999958436
Stopped at epoch:  6
1/1 [==============================] - 0s 16ms/step
+++++++++++ [18_9] | 24 ++++++++++++++++++
Elapsed time:  0.28949107799962803
Stopped at epoch:  6
1/1 [==============================] - 0s 15ms/step
+++++++++++ [18_9] | 25 ++++++++++++++++++
Elapsed time:  0.3533479890002127
Stopped at epoch:  6
1/1 [==============================] - 0s 17ms/step
